<font color='red'><b>!!!首先将paddlenlp更新到最新版本!!!</b></font>

In [1]:
!pip install --upgrade paddlenlp

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 430kB 13.7MB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


In [2]:
import paddlenlp

print(paddlenlp.__version__)

2.0.2


# 下载MRSA-NER数据集

In [3]:
# 下载 msra-ner 数据集到当前目录
!wget https://paddlenlp.bj.bcebos.com/datasets/msra_ner.tar.gz

--2021-06-12 02:55:24--  https://paddlenlp.bj.bcebos.com/datasets/msra_ner.tar.gz
Resolving paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c00:6c21:10ad:0:ff:b00e:67d
Connecting to paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3743966 (3.6M) [application/octet-stream]
Saving to: ‘msra_ner.tar.gz’

msra_ner.tar.gz     100%[===================>]   3.57M  14.7MB/s    in 0.2s    

2021-06-12 02:55:25 (14.7 MB/s) - ‘msra_ner.tar.gz’ saved [3743966/3743966]



In [4]:
# 解压msra-ner数据集
!tar -zxvf ./msra_ner.tar.gz

msra_ner/
msra_ner/train.tsv
msra_ner/label_map.json
msra_ner/test.tsv


# 数据集描述

解压后的msra_ner目录下有三个文件：train.tsv, test.tsv, label_map.json

其中 train.tsv 和 test.tsv 的每一行为一个样本，样本格式为：

tokens	labels

tokens 为需要标注的样本，labels 为标注信息，二者由制表符(\t)隔开。tokens 中的每一个字和labels中的每一个标注由STX（即\002）字符隔开。

label_map.json，即标记字典文件，存放了每一标记和标记ID的映射：




MSRA-NER数据集中，训练集包含45000个样本，测试集包含3442个样本。

数据集包含三类实体：人物(Person), 组织机构（Organization），地点（Location）和其他（Other），对应的标记简写为：PER，ORG，LOC和O。

数据集采用BIO标注的方式，label_map.json文件，即标记字典文件，存放了每一个标记和标记ID的映射：

```
    {
      "B-PER": 0,
      "I-PER": 1,
      "B-ORG": 2,
      "I-ORG": 3,
      "B-LOC": 4,
      "I-LOC": 5,
      "O": 6
    }
```

每一个标记的含义如下：

|标记|含义|
|---|---|
|B-PER|人名的开始字符|
|I-PER|人名的非开始字符|
|B-ORG|组织机构名的开始字符|
|I-ORG|组织机构名的非开始字符|
|B-LOC|地点/位置名的开始字符|
|I-LOC|地点/位置名的非开始字符|
|O|非命名实体部分|


# 划分数据集

训练集包含 45000个样本，测试集包含3442个样本，由于没有验证集，因此我们需要从训练集中划分出与测试集规模相当的验证集出来，用于在训练过程中对模型进行评估。测试集保持不变。我们选择从训练集中随机抽取3000个样本作为验证集。

|数据集|样本数|
|---|---|
|training set| 42000|
|dev set|3000|
|test set|3442|

In [5]:
def train_test_split(train_file, proportion=3000):
    """
    proportion: int or float
    """
    with open(train_file, 'r', encoding='utf-8') as fRead:
        train_data = []
        for line in fRead.readlines():
            train_data.append(line.strip('\n'))
        
        data_id = list(range(len(train_data)))

        train_len = len(data_id)
        
        if isinstance(proportion , int):
            test_split_len = proportion
        elif isinstance(proportion, float):
            test_split_len = int(train_len * proportion)
        else:
            raise ValueError('proportion must be int or float!')


        import random
        random.seed(5233)
        random.shuffle(data_id)

        test_split_data = [train_data[idx] for idx in data_id[:test_split_len]]
        train_split_data = [train_data[idx] for idx in data_id[test_split_len:]]

        import os
        data_dir = './msra_ner/data/msra'
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)

        train_file = os.path.join(data_dir, 'train.tsv')
        test_file = os.path.join(data_dir, 'dev.tsv')

        with open(train_file, 'w', encoding='utf-8') as fWriter1:
            for train_line in train_split_data:
                fWriter1.write(train_line + '\n')

        with open(test_file, 'w', encoding='utf-8') as fWriter2:
            for test_line in test_split_data:
                fWriter2.write(test_line + '\n')


In [6]:
train_test_split('./msra_ner/train.tsv')

In [7]:
!cp ./msra_ner/test.tsv ./msra_ner/data/msra/test.tsv
!cp ./msra_ner/label_map.json ./msra_ner/data/msra/label_map.json

In [8]:
train_data = []
with open('./msra_ner/data/msra/train.tsv', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        train_data.append(line)

print(len(train_data))



42000


# 加载数据集

In [9]:
import os

from paddlenlp.datasets import load_dataset


def read(data_path):
    with open(data_path, 'r', encoding='utf-8') as fRead:
        for line in fRead:
            tokens, labels = line.strip('\n').split('\t')
            tokens = tokens.split('\002')  # \002 即 STX 字符
            labels = labels.split('\002')
            yield {'tokens': tokens, 'labels': labels}

data_dir = './msra_ner/data/msra'
train_file = os.path.join(data_dir, 'train.tsv')
dev_file = os.path.join(data_dir, 'dev.tsv')
test_file = os.path.join(data_dir, 'test.tsv')

train_dataset = load_dataset(read, data_path=train_file, lazy=False)
dev_dataset = load_dataset(read, data_path=dev_file, lazy=False)
test_dataset = load_dataset(read, data_path=test_file, lazy=False)


In [10]:
for idx, ex in enumerate(train_dataset):
    if idx < 3:
        print(ex)

{'tokens': ['苗', '苗', '妈', '妈', '给', '孩', '子', '以', '信', '任', '和', '期', '待', '，', '并', '且', '巧', '妙', '地', '从', '孩', '子', '力', '所', '能', '及', '的', '日', '常', '生', '活', '小', '事', '出', '发', '，', '让', '孩', '子', '在', '劳', '动', '中', '体', '验', '生', '活', '的', '苦', '与', '乐', '。'], 'labels': ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}
{'tokens': ['郎', '青', '山', '亲', '自', '抓', '产', '供', '销', '和', '技', '术', '改', '造', '，', '副', '总', '经', '理', '分', '管', '财', '务', '、', '后', '勤', '和', '生', '产', '调', '度', '，', '其', '他', '人', '负', '责', '技', '术', '监', '督', '。'], 'labels': ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [11]:
import json


with open('./msra_ner/data/msra/label_map.json', 'r', encoding='utf-8') as fRead:
    label_vocab = json.load(fRead)

print(label_vocab)

print(len(label_vocab))

{'B-PER': 0, 'I-PER': 1, 'B-ORG': 2, 'I-ORG': 3, 'B-LOC': 4, 'I-LOC': 5, 'O': 6}
7


# 构建BERT输入特征

In [12]:
from paddlenlp.transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

def convert_example(example, tokenizer, label_vocab, max_seq_len=512):
    tokens, labels = example['tokens'], example['labels']
    
    tokenized_inputs = tokenizer(tokens, return_length=True, is_split_into_words=True, max_seq_len=max_seq_len)

    labels = [label_vocab[label] for label in labels]
    if len(labels) + 2 > max_seq_len:
        labels = labels[: max_seq_len - 2] 

    tokenized_inputs['labels'] = [label_vocab['O']] + labels + [label_vocab['O']] 

    return tokenized_inputs['input_ids'], tokenized_inputs['token_type_ids'], tokenized_inputs['seq_len'], tokenized_inputs['labels']

for idx, example in enumerate(train_dataset):
    if idx < 1:
        print(convert_example(example, tokenizer, label_vocab))

[2021-06-12 02:55:52,819] [    INFO] - Downloading bert-base-chinese-vocab.txt from https://paddle-hapi.bj.bcebos.com/models/bert/bert-base-chinese-vocab.txt
100%|██████████| 107/107 [00:00<00:00, 3318.79it/s]


([101, 5728, 5728, 1968, 1968, 5314, 2111, 2094, 809, 928, 818, 1469, 3309, 2521, 8024, 2400, 684, 2341, 1975, 1765, 794, 2111, 2094, 1213, 2792, 5543, 1350, 4638, 3189, 2382, 4495, 3833, 2207, 752, 1139, 1355, 8024, 6375, 2111, 2094, 1762, 1227, 1220, 704, 860, 7741, 4495, 3833, 4638, 5736, 680, 727, 511, 102], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 54, [6, 0, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])


In [13]:
from functools import partial


trans_func = partial(convert_example, tokenizer=tokenizer, label_vocab=label_vocab)

train_dataset.map(trans_func)
dev_dataset.map(trans_func)
test_dataset.map(trans_func)

In [14]:
for idx, example in enumerate(train_dataset):
    if idx < 3:
        print(example)

([101, 5728, 5728, 1968, 1968, 5314, 2111, 2094, 809, 928, 818, 1469, 3309, 2521, 8024, 2400, 684, 2341, 1975, 1765, 794, 2111, 2094, 1213, 2792, 5543, 1350, 4638, 3189, 2382, 4495, 3833, 2207, 752, 1139, 1355, 8024, 6375, 2111, 2094, 1762, 1227, 1220, 704, 860, 7741, 4495, 3833, 4638, 5736, 680, 727, 511, 102], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 54, [6, 0, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])
([101, 6947, 7471, 2255, 779, 5632, 2831, 772, 897, 7218, 1469, 2825, 3318, 3121, 6863, 8024, 1199, 2600, 5307, 4415, 1146, 5052, 6568, 1218, 510, 1400, 1249, 1469, 4495, 772, 6444, 2428, 8024, 1071, 800, 782, 6566, 6569, 2825, 3318, 4664, 4719, 511, 102], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# 构建数据集加载器 DataLoader

In [15]:
from paddle.io import DataLoader
from paddle.io import DistributedBatchSampler
from paddle.io import BatchSampler
from paddlenlp.data import Pad
from paddlenlp.data import Stack
from paddlenlp.data import Tuple


pad_label_id = -1
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id), # token_type_ids
    Stack(), # seq_len
    Pad(axis=0, pad_val=pad_label_id)  # labels
): [data for data in fn(samples)]

train_batch_sampler = DistributedBatchSampler(
    dataset=train_dataset, 
    batch_size=32, 
    shuffle=True
)
train_data_loader = DataLoader(
    dataset=train_dataset, 
    batch_sampler=train_batch_sampler, 
    collate_fn=batchify_fn, 
    return_list=True
)

dev_batch_sampler = BatchSampler(
    dataset=dev_dataset,
    batch_size=32,
    shuffle=False
)
dev_data_loader = DataLoader(
    dataset=dev_dataset,
    batch_sampler=dev_batch_sampler,
    collate_fn=batchify_fn,
    return_list=True
)

test_batch_sampler = BatchSampler(
    dataset=test_dataset,
    batch_size=32,
    shuffle=False
)
test_data_loader = DataLoader(
    dataset=test_dataset,
    batch_sampler=test_batch_sampler,
    collate_fn=batchify_fn,
    return_list=True
)


# 创建模型 ：加载 BERT 预训练模型

In [16]:
from paddlenlp.transformers import BertForTokenClassification


model = BertForTokenClassification.from_pretrained('bert-base-chinese', num_classes=len(label_vocab))

[2021-06-12 02:56:06,020] [    INFO] - Downloading http://paddlenlp.bj.bcebos.com/models/transformers/bert/bert-base-chinese.pdparams and saved to /home/aistudio/.paddlenlp/models/bert-base-chinese
[2021-06-12 02:56:06,071] [    INFO] - Downloading bert-base-chinese.pdparams from http://paddlenlp.bj.bcebos.com/models/transformers/bert/bert-base-chinese.pdparams
100%|██████████| 696494/696494 [00:10<00:00, 66666.58it/s]
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for classifier.bias. classifier.bias is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))


# 模型训练与评估


In [19]:
from paddle.nn import CrossEntropyLoss
from paddle.optimizer import AdamW
from paddlenlp.metrics import ChunkEvaluator

loss_fn = CrossEntropyLoss(ignore_index=pad_label_id)
metric = ChunkEvaluator(label_list=label_vocab.keys(), suffix=False)
optimizer = AdamW(learning_rate=2e-5, parameters=model.parameters())


In [20]:
import paddle
import numpy as np


@paddle.no_grad()
def evaluate(model,loss_fn, metric, data_loader):
    model.eval()
    metric.reset()

    losses = []

    for batch_data in data_loader:

        input_ids, token_type_ids, length, labels = batch_data

        logits = model(input_ids, token_type_ids)

        loss = loss_fn(logits, labels)
        losses.append(loss.numpy())

        preds = paddle.argmax(logits, axis=-1)
        n_infer, n_label, n_correct = metric.compute(None, length, preds, labels)
        metric.update(n_infer.numpy(), n_label.numpy(), n_correct.numpy())
        precission, recall, f1_score = metric.accumulate()

    model.train()

    return np.mean(losses), precission, recall, f1_score


In [21]:
import paddle
import time
import os


global_step = 0
epochs = 1
best_f1_score = 0.0
best_precission = 0.0
best_recall = 0.0

print_every_step = 10
evaluate_every_step = 100

save_dir = os.path.join('./data', 'checkpoints')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_param_path = os.path.join(save_dir, 'bets_model_state.pdparams')

tic_train = time.time()

for epoch in range(epochs):
    for step, batch_data in enumerate(train_data_loader):
        
        input_ids, token_type_ids, length, labels = batch_data
        logits = model(input_ids, token_type_ids)

        loss = loss_fn(logits, labels)

        preds = paddle.argmax(logits, axis=-1)
        n_infer, n_label, n_correct = metric.compute(None, length, preds, labels)
        metric.update(n_infer.numpy(), n_label.numpy(), n_correct.numpy())
        precission, recall, f1_score = metric.accumulate()

        global_step += 1

        if global_step % print_every_step == 0:
            print('global_step: %d, epoch: %d, batch: %d, loss: %.5f, precission: %.5f, recall: %.5f, f1_score: %.5f, speed: %.2f step/s' % (
                global_step, epoch, step, loss.numpy(), precission, recall, f1_score, print_every_step / (time.time() - tic_train)
            ))
            tic_train = time.time()
        
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        if global_step % evaluate_every_step == 0:
            loss, precission, recall, f1_score = evaluate(model, loss_fn, metric, dev_data_loader)
            print('eval dev loss: %.5f, precission: %.5f, recall: %.5f, f1_score: %.5f' % (
                loss, precission, recall, f1_score
            ))
            if f1_score > best_f1_score and precission > best_precission and recall > best_recall:
                
                best_f1_score = f1_score
                best_precission = precission 
                best_recall = best_recall:

                print('save model at global step : %d, best_precission: %.5f, best_recall: %.5f, best val f1_score: %.5f' % (
                    global_step, best_precission, best_recall, best_f1_score
                ))

                paddle.save(model.state_dict(), save_param_path)
                tokenizer.save_pretrained(save_dir)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
[2021-06-12 03:00:04,273] [ WARNING] - Compatibility Warning: The params of ChunkEvaluator.compute has been modified. The old version is `inputs`, `lengths`, `predictions`, `labels` while the current version is `lengths`, `predictions`, `labels`.  Please update the usage.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprec

global_step: 10, epoch: 0, batch: 9, loss: 0.37573, precission: 0.00213, recall: 0.00668, f1_score: 0.00323, speed: 4.27 step/s
global_step: 20, epoch: 0, batch: 19, loss: 0.39082, precission: 0.00213, recall: 0.00325, f1_score: 0.00257, speed: 3.85 step/s
global_step: 30, epoch: 0, batch: 29, loss: 0.22917, precission: 0.00258, recall: 0.00281, f1_score: 0.00269, speed: 4.85 step/s
global_step: 40, epoch: 0, batch: 39, loss: 0.23283, precission: 0.03394, recall: 0.03645, f1_score: 0.03515, speed: 4.72 step/s
global_step: 50, epoch: 0, batch: 49, loss: 0.11737, precission: 0.11885, recall: 0.13220, f1_score: 0.12517, speed: 4.56 step/s
global_step: 60, epoch: 0, batch: 59, loss: 0.09801, precission: 0.19408, recall: 0.21962, f1_score: 0.20606, speed: 4.06 step/s
global_step: 70, epoch: 0, batch: 69, loss: 0.11979, precission: 0.26075, recall: 0.29829, f1_score: 0.27826, speed: 3.65 step/s
global_step: 80, epoch: 0, batch: 79, loss: 0.11220, precission: 0.29760, recall: 0.34032, f1_scor

# 预测

In [24]:
def predict(model, data_loader, ds, label_vocab):
    pred_list = []
    len_list = []
    for input_ids, seg_ids, lens, labels in data_loader:
        logits = model(input_ids, seg_ids)
        pred = paddle.argmax(logits, axis=-1)
        pred_list.append(pred.numpy())
        len_list.append(lens.numpy())

    return pred_list, len_list

In [25]:
pred_list, len_list = predict(model, test_data_loader, test_dataset, label_vocab)

In [27]:
label_vocab

{'B-PER': 0,
 'I-PER': 1,
 'B-ORG': 2,
 'I-ORG': 3,
 'B-LOC': 4,
 'I-LOC': 5,
 'O': 6}

In [28]:
import numpy as np
np.set_printoptions(threshold = 1e6)

In [33]:
pred_list[3][4]

array([6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5, 5, 5, 3, 3, 6,
       6, 6, 6, 6, 4, 5, 5, 5, 5, 5, 6, 6, 0, 1, 1, 6, 6, 6, 6, 6, 4, 5,
       5, 5, 6, 6, 0, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5, 6, 0, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 4, 6, 4, 5,
       6, 5, 6, 6, 0, 0, 1, 1, 6, 6, 6, 6, 6, 4, 4, 6, 5, 6, 0, 0, 1, 1,
       6, 1, 6, 6, 6, 6, 6, 6, 6, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 4, 5, 5, 6, 0, 1, 1, 6, 6, 6, 6, 6, 6, 4, 5, 5, 6, 6, 0, 0,
       0, 0, 1, 6, 6, 6, 6, 6, 4, 4, 5, 5, 5, 5, 6, 6, 0, 0, 0, 0, 1, 1,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, 3, 5, 5, 5, 5, 5, 5, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 5], dtype=int64)

In [34]:
def predict(model, data_loader, ds, label_vocab):
    pred_list = []
    len_list = []
    for input_ids, seg_ids, lens, labels in data_loader:
        logits = model(input_ids, seg_ids)
        pred = paddle.argmax(logits, axis=-1)
        pred_list.append(pred.numpy())
        len_list.append(lens.numpy())
    preds = parse_decodes(ds, pred_list, len_list, label_vocab)
    return preds

def parse_decodes(ds, decodes, lens, label_vocab):
    decodes = [x for batch in decodes for x in batch]  # [[]]
    lens = [x for batch in lens for x in batch]        # []
    id_label = dict(zip(label_vocab.values(), label_vocab.keys()))

    outputs = []
    for idx, end in enumerate(lens):
        sent = ds.data[idx]['tokens'][:end]
        tags = [id_label[x] for x in decodes[idx][1:end]]
        sent_out = []
        tags_out = []
        words = ""
        for s, t in zip(sent, tags):
            if t.startswith('B-') or t == 'O':
                if len(words):
                    sent_out.append(words)
                if t.startswith('B-'):
                    tags_out.append(t.split('-')[1])
                else:
                    tags_out.append(t)
                words = s
            else:
                words += s
        if len(sent_out) < len(tags_out):
            sent_out.append(words)
        outputs.append(''.join(
            [str((s, t)) for s, t in zip(sent_out, tags_out)]))
    return outputs



In [ ]:
# save_dir = os.path.join('./data', 'checkpoints')

# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
# save_param_path = os.path.join(save_dir, 'bets_model_state.pdparams')

# state_dict = paddle.load(save_param_path)
# model.set_dict(state_dict)    

In [35]:
preds = predict(model, test_data_loader, test_dataset, label_vocab)
file_path = "msra_ner_results.txt"
with open(file_path, "w", encoding="utf8") as fout:
    fout.write("\n".join(preds))
# Print some examples
print(
    "The results have been saved in the file: %s, some examples are shown below: "
    % file_path)
print("\n".join(preds[:10]))

The results have been saved in the file: msra_ner_results.txt, some examples are shown below: 
('中共中央', 'ORG')('致', 'O')('中国致公党十一大', 'ORG')('的', 'O')('贺', 'O')('词', 'O')('各', 'O')('位', 'O')('代', 'O')('表', 'O')('、', 'O')('各', 'O')('位', 'O')('同', 'O')('志', 'O')('：', 'O')('在', 'O')('中国致公党第十一次全国代表大会', 'ORG')('隆', 'O')('重', 'O')('召', 'O')('开', 'O')('之', 'O')('际', 'O')('，', 'O')('中国共产党中央委员会', 'ORG')('谨', 'O')('向', 'O')('大', 'O')('会', 'O')('表', 'O')('示', 'O')('热', 'O')('烈', 'O')('的', 'O')('祝', 'O')('贺', 'O')('，', 'O')('向', 'O')('致公党', 'ORG')('的', 'O')('同', 'O')('志', 'O')('们', 'O')('致', 'O')('以', 'O')('亲', 'O')('切', 'O')('的', 'O')('问', 'O')('候', 'O')('！', 'O')
('在', 'O')('过', 'O')('去', 'O')('的', 'O')('五', 'O')('年', 'O')('中', 'O')('，', 'O')('致公党', 'ORG')('在', 'O')('邓小平', 'PER')('理', 'O')('论', 'O')('指', 'O')('引', 'O')('下', 'O')('，', 'O')('遵', 'O')('循', 'O')('社', 'O')('会', 'O')('主', 'O')('义', 'O')('初', 'O')('级', 'O')('阶', 'O')('段', 'O')('的', 'O')('基', 'O')('本', 'O')('路', 'O')('线', 'O')('，', 'O')(

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 